In [1]:
%install_ext http://raw.github.com/jrjohansson/version_information/master/version_information.py
%load_ext version_information
%reload_ext version_information
%version_information numpy, scipy, matplotlib, pandas

/home/ubuntu/anaconda2/lib/python2.7/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension as a python package.
  "as a python package.", UserWarning)


Installed version_information.py. To use it, type:
  %load_ext version_information


Software versions
Python 2.7.11 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 4.1.2
OS Linux 3.13.0 74 generic x86_64 with debian jessie sid
numpy 1.10.4
scipy 0.17.0
matplotlib 1.5.1
pandas 0.18.0
Wed Jun 08 02:39:33 2016 UTC

In [2]:
# Couldn't find what to do with 'magic' imports at the top

%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format='retina' 

# Add this to python2 code to make life easier
from __future__ import absolute_import, division, print_function

from itertools import combinations
import string

from IPython.display import IFrame, HTML, YouTubeVideo
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns

sns.set();
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("darkgrid")
sns.set_context("poster", font_scale=1.3)

In [3]:
# talk to postgresql
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd

In [4]:
dbname = 'stackoverflow'
username = 'jojo'
pswd = 'iAmPass'

con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

# Posts

- Id
- PostTypeId (listed in the PostTypes table)
  - Question (1)
  - Answer (2)
  - Orphaned tag wiki
  - Tag wiki excerpt
  - Tag wiki
  - Moderator nomination
  - "Wiki placeholder" (seems to only be the election description)
  - Privilege wiki
- AcceptedAnswerId (only present if PostTypeId is 1)
- ParentID (only present if PostTypeId is 2)
- CreationDate
- DeletionDate (only non-null for the SEDE PostsWithDeleted table. Deleted posts are not present on Posts. Column not present on data dump.)
- Score
- ViewCount (nullable)
- Body (as rendered HTML, not Markdown)
- OwnerUserId (only present if user has not been deleted; always -1 for tag wiki entries, i.e. the community user owns them)
- OwnerDisplayName (nullable)
- LastEditorUserId (nullable)
- LastEditorDisplayName (nullable)
- LastEditDate="2009-03-05T22:28:34.823" - the date and time of the most recent edit to the post (nullable)
- LastActivityDate="2009-03-11T12:51:01.480" - the date and time of the most recent activity on the post. For a question, this could be the post being edited, a new answer was posted, a bounty was started, etc.
- Title (nullable)
- Tags (nullable)
- AnswerCount (nullable)
- CommentCount
- FavoriteCount
- ClosedDate (present only if the post is closed)
- CommunityOwnedDate (present only if post is community wikied)

## Clean the Posts
1. Remove the records that are neight questions nor answers

In [5]:
# query:
sql_query = """
SELECT * FROM posts 
where posttypeid = 1 or posttypeid = 2
limit 4000;
"""
posts = pd.read_sql_query(sql_query, con)

In [6]:
posts.head()

,id,posttypeid,acceptedanswerid,parentid,creationdate,score,viewcount,body,owneruserid,lasteditoruserid,lasteditordisplayname,lasteditdate,lastactivitydate,title,tags,answercount,commentcount,favoritecount,closeddate,communityowneddate
0,15691568,2,NaN,15691531.0,2013-03-28 20:30:23.450,3,NaN,"<p>They're a backup copy, and can be very usef...",173292.0,NaN,None,None,2013-03-28 20:30:23.450,None,None,NaN,0,NaN,None,None
1,15691569,1,15694294.0,NaN,2013-03-28 20:30:24.960,1,3653.0,<p>I try to create a web page with a fixed nav...,1937219.0,NaN,None,None,2013-03-29 00:02:30.490,JavaScript issue with scrollTo() in Chrome,<javascript><google-chrome><scrollto>,1.0,1,NaN,None,None
2,15691570,2,NaN,15624275.0,2013-03-28 20:30:26.577,1,NaN,<p>If you are using the same Animation object ...,1234408.0,NaN,None,None,2013-03-28 20:30:26.577,None,None,NaN,1,NaN,None,None
3,15691571,2,NaN,709801.0,2013-03-28 20:30:27.107,0,NaN,<p>I like the example given by Active State us...,2055968.0,NaN,None,None,2013-03-28 20:30:27.107,None,None,NaN,0,NaN,None,None
4,15691572,1,15691753.0,NaN,2013-03-28 20:30:28.577,0,1647.0,<p>I have three files. I want to compare colu...,2100945.0,2100945.0,None,2013-03-29 00:35:12.980000,2013-03-29 00:35:12.980,"Compare values from two csv files, append valu...",<python><csv><python-2.7>,1.0,0,NaN,None,None


# Votes
- Id
- PostId
- VoteTypeId (listed in the VoteTypes table)
  1. AcceptedByOriginator
  2. UpMod (AKA upvote)
  3. DownMod (AKA downvote)
  4. Offensive
  5. Favorite (UserId will also be populated)
  6. Close (since 2013-06-25 close votes are ONLY stored in the PostHistory table)
  7. Reopen
  8. BountyStart (UserId and BountyAmount will also be populated)
  9. BountyClose (BountyAmount will also be populated)
  10. Deletion
  11. Undeletion
  12. Spam
  15. ModeratorReview
  16. ApproveEditSuggestion
- UserId (only present if VoteTypeId is 5 or 8)
- CreationDate (time data is purposefully removed to protect user privacy)
- BountyAmount (only present if VoteTypeId is 8 or 9)

In [7]:
sql_query = """
SELECT * FROM votes limit 2;
"""
votes = pd.read_sql_query(sql_query, con)

In [8]:
votes

,id,postid,votetypeid,userid,creationdate,bountyamount
0,1,1,2,None,2008-07-31,None
1,2,3,2,None,2008-07-31,None


# Users

- Id
- Reputation
- CreationDate
- DisplayName
- LastAccessDate
- WebsiteUrl
- Location
- AboutMe
- Views
- UpVotes (how many upvotes the user has cast)
- DownVotes
- EmailHash (now always blank)
- AccountId (StackExchange Network profile Id of the user)
- Age

In [9]:
sql_query = """
SELECT * FROM users limit 2;
"""
users= pd.read_sql_query(sql_query, con)

In [10]:
users

,id,reputation,creationdate,displayname,lastaccessdate,websiteurl,location,aboutme,views,upvotes,downvotes,profileimageurl,age,accountid
0,-1,1,2008-07-31 00:00:00.000,Community,2008-08-26 00:16:53.810,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",649,143094,643540,None,NaN,-1
1,1,36925,2008-07-31 14:22:31.287,Jeff Atwood,2016-03-04 01:10:16.363,http://www.codinghorror.com/blog/,"El Cerrito, CA","<p><a href=""http://www.codinghorror.com/blog/a...",209166,3275,1300,https://www.gravatar.com/avatar/51d623f33f8b83...,46.0,1


## TODO Remove the user with id = -1

# Tags
- Id
- TagName
- Count
- ExcerptPostId
- WikiPostId

In [11]:
sql_query = """
SELECT * FROM tags limit 2;
"""
tags = pd.read_sql_query(sql_query, con)

In [12]:
tags

,id,tagname,count,excerptpostid,wikipostid
0,1,.net,227675,3624959,3607476
1,2,html,511091,3673183,3673182


# Badges
- Id
- UserId, e.g.: "420"
- Name, e.g.: "Teacher"
- Date, e.g.: "2008-09-15T08:55:03.923"
- Class
- Gold
- Silver
- Bronze
- TagBased, true if badge is for a tag, otherwise it is a named badge

In [13]:
sql_query = """
SELECT * FROM badges limit 2;
"""
badges = pd.read_sql_query(sql_query, con)

In [14]:
badges

,id,userid,name,date
0,2714119,1150276,Student,2012-01-15 11:04:23.393
1,2714120,326874,Critic,2012-01-15 11:09:14.597
